In [5]:
import numpy as np
import pandas as pd
import duckdb

In [2]:
# %pip install openpyxl

In [3]:
# Load Electric Vehicle dataset
ev_df = pd.read_csv('Electric_Vehicle_Population_Data.csv', encoding='ISO-8859-1')
print(ev_df.shape)
ev_df.head()

(177866, 17)


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJSA1E4XK,King,Seattle,WA,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,POINT (-122.34848 47.632405),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,5YJSA1E27G,King,Issaquah,WA,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,POINT (-122.03646 47.534065),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,5YJYGDEE5M,Kitsap,Suquamish,WA,98392.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23.0,205138552,POINT (-122.55717 47.733415),PUGET SOUND ENERGY INC,5.303594e+10


For the electric vehicle dataset, we removed any data that contains null or no values so it does not affect our analysis. We removed columns 2020 Census Tract and Vehicle Location since these columns do not provide use for our analysis. We also dropped any vehicles that has an electric range less than equal to 20 since such a small electric range seems nonsensical and might skew our data analysis.

In [4]:
# Dropped any data that contains null or no values
ev_df = ev_df.dropna(how="all")
ev_df = ev_df.dropna(axis=1, how="all")
columns_to_remove = ['2020 Census Tract', 'Vehicle Location']
ev_df = ev_df.drop(columns=columns_to_remove)
ev_df = ev_df[ev_df['Electric Range'] > 20]
ev_df['Model Year'] = ev_df['Model Year'].astype(int)
ev_df.head()
ev_df.to_csv('Electric_Vehicle_Data.csv', index=False)

In [5]:
# Load Median Income dataset
income_df = pd.read_excel('median_household_income_estimates.xlsx', sheet_name='Prelim2022Proj2023 (PV)(Final)',
                         engine='openpyxl')
print(income_df.shape)
income_df.head(15)

(59, 36)


,Washington State Median Household Income Estimates by County: 1989 to 2021; Preliminary estimates for 2022 and Projections for 2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,and Projections for 2023 in Current Dollars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Census,Estimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prelim. Estimate,Projection
2,NaN,1989,1990,1991.000,1992.000,1993.000,1994.000,1995.000,1996.000,1997.000,...,2014.000000,2015.00,2016.000000,2017.000000,2018.000000,2019.0,2020.0,2021.0,2022,2023
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Washington,31183,33417.061,34379.118,35881.524,36678.568,37894.965,38996.782,40568.226,42399.362,...,60153.000000,63439.20,65500.290875,69287.868831,72297.030172,78674.0,80319.0,84155.0,91255,94307.535042
5,Adams,24604,28024.074,27300.700,28328.127,31277.360,29537.150,29603.720,31806.107,31794.792,...,45711.821556,47645.73,49501.385091,48848.793830,52870.057114,53535.0,56421.0,57405.0,62410,64497.652315
6,Asotin,22897,24206.219,25110.440,26250.554,27358.886,27872.895,28288.256,29812.595,31498.836,...,43367.862197,46107.27,47020.113312,51766.880459,50746.366302,54776.0,53377.0,56438.0,65625,67820.196013
7,Benton,32593,35011.606,37093.354,39227.763,41153.190,42789.890,42539.628,42816.605,44056.502,...,63157.401995,62071.02,62282.049221,63501.707462,67912.297802,72847.0,75882.0,77339.0,85144,87992.118389
8,Chelan,24312,25833.442,27592.308,28745.697,30148.302,31546.570,32163.776,33918.164,35661.808,...,50825.030213,53067.96,55109.142422,60790.913046,60747.403036,59838.0,61546.0,64700.0,81697,84429.814151
9,Clallam,25434,27329.22,27861.112,28577.986,29340.212,29950.640,31162.518,32558.772,34769.840,...,45453.815015,46240.92,48187.110514,47766.585450,55664.085098,57571.0,54712.0,62623.0,66693,68923.92126


As we can see from the dataframe, the first few rows of the dataframe contains NaN values or untamed column headings. The row we really want to be our column headings is row #2. We dropped row #0, #1, and #3, and made row #2 the dataframe heading. We removed columns from 1989 to 2020 since any data before 2021 might not accurately represent each county's median income in Washington State.

In [6]:
income_df = income_df.drop(index=[0, 1, 3])
income_df.columns = income_df.iloc[0]
income_df = income_df[1:]
income_df.reset_index(drop=True, inplace=True)
income_df.rename(columns={income_df.columns[0]: "County"}, inplace=True)
income_df.columns = income_df.columns.astype(str)
columns_to_keep = ['County', '2021.0', '2022', '2023']
income_df = income_df[columns_to_keep]
# Made sure to rename any columns from a string float to a string int
income_df.rename(columns={'2021.0': '2021'}, inplace=True)
income_df.dropna(inplace=True)
# Made all columns header's type into an integer
for col in ['2021', '2022', '2023']:
    income_df[col] = income_df[col].astype(float).round(0).astype(int)
income_df
income_df.to_csv('Median_Household_Income.csv', index=False)

In [7]:
# Load Population dataset
population_df = pd.read_csv('Washington Population.csv', encoding='ISO-8859-1')
print(population_df.shape)
population_df.head()

(409, 39)


,SEQUENCE,FILTER,COUNTY,JURISDICTION,POP_1990,POP_1991,POP_1992,POP_1993,POP_1994,POP_1995,...,POP_2015,POP_2016,POP_2017,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,POP_2024
0,1,1,Adams,Adams County,13603.0,13823.0,14063.0,14335.0,14679.0,15030.0,...,19451,19643,20068,20200,20335,20613,20900,21100,21200,21475
1,2,2,Adams,Unincorporated Adams County,6466.0,6698.0,6776.0,7009.0,7162.0,7303.0,...,9093,9205,9250,9301,9375,9472,9575,9575,9585,9628
2,3,3,Adams,Incorporated Adams County,7137.0,7125.0,7287.0,7326.0,7517.0,7727.0,...,10358,10438,10818,10899,10960,11141,11325,11525,11615,11847
3,4,4,Adams,Hatton,71.0,80.0,81.0,82.0,83.0,84.0,...,90,87,85,82,82,79,80,80,80,167
4,5,4,Adams,Lind,472.0,400.0,523.0,435.0,452.0,451.0,...,565,556,556,534,535,535,535,535,535,535


We removed aan data that contains null or no values so it does not affect our analysis. We removed columns SEQUENCE and FILTER since these columns would not help our analysis. We also removed any data from 1990 to 2020 since any data before 2021 might not accurately represent each county's population in Washington State. We also removed year 2024 since our median income dataset only contains data up to 2023.

In [8]:
population_df = population_df.dropna(how="all")
population_df = population_df.dropna(axis=1, how="all")
columns_to_remove = ['SEQUENCE', 'FILTER']
population_df = population_df.drop(columns=columns_to_remove)
years_to_remove = [col for col in population_df.columns if col.startswith("POP_") and 1990 <= int(col.split('_')[1]) <= 2020]
population_df = population_df.drop(columns=years_to_remove)
population_df = population_df.drop(columns= ['POP_2024'])
population_df.head()

,COUNTY,JURISDICTION,POP_2021,POP_2022,POP_2023
0,Adams,Adams County,20900,21100,21200
1,Adams,Unincorporated Adams County,9575,9575,9585
2,Adams,Incorporated Adams County,11325,11525,11615
3,Adams,Hatton,80,80,80
4,Adams,Lind,535,535,535


Since each row represents either a county or a specific city within a county, we would only want data pertaining to a county. We removed any rows that were not specific to a county. All county rows mainly starts with the county's name and then the word County. We removed any rows that did not follow this pattern.

In [9]:
query = """
    SELECT COUNTY, POP_2021, POP_2022, POP_2023
    FROM population_df
    WHERE LOWER(JURISDICTION) LIKE '%county%'
      AND LOWER(JURISDICTION) NOT LIKE '%unincorporated%'
      AND LOWER(JURISDICTION) NOT LIKE '%incorporated%';
"""
population_df = duckdb.query(query).to_df()
population_df
population_df.to_csv('Washington_Population.csv', index=False)

We aggregated the data from the electric vehicle dataset with the median income dataset and population dataset based on the county. We decided to only use year 2023 since that is the closest year to the current year, 2024.

In [10]:
query = """
    SELECT 
        ev_df.*,
        income_df."2023" AS income_2023,
        population_df."POP_2023" AS population_2023
    FROM ev_df
    LEFT JOIN income_df
        ON ev_df.County = income_df.County
    LEFT JOIN population_df
        ON ev_df.County = population_df.COUNTY
"""
result_df = duckdb.query(query).to_df()
result_df.head()
result_df.to_csv('Combined_Data.csv', index=False)

In [6]:
# Load the combined data dataset
df = pd.read_csv('Combined_Data.csv', encoding='ISO-8859-1')
print(df.shape)
df.head()

(77114, 17)


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Electric Utility,income_2023,population_2023
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),119926.0,2347800.0
1,5YJSA1E4XK,King,Seattle,WA,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),119926.0,2347800.0
2,5YJSA1E27G,King,Issaquah,WA,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),119926.0,2347800.0
3,3FA6P0SU8H,Thurston,Yelm,WA,98597.0,2017,FORD,FUSION,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21,0,2.0,122057736,PUGET SOUND ENERGY INC,91522.0,303400.0
4,1N4AZ0CP2D,Yakima,Yakima,WA,98903.0,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75,0,14.0,150126840,PACIFICORP,65167.0,261200.0


We decided to aggregate the dataset by county where each row would represent a county. We also found the total EV count, average electric range, average model year, popular brand, popular model, popular EV type, BEV proportion, median income and population for each county.

In [7]:
aggregated_df = duckdb.sql("""
    SELECT 
        County,
        COUNT(*) AS "Total EV Count",  -- Total count of all EVs in the county
        AVG("Electric Range") AS "Average Electric Range", -- Average Electric Range in the county
        AVG("Model Year") AS "Average Model Year", -- Average EV Model Year in the county
        MODE("Make") AS "Popular Brand", -- Most popular EV Brand in the county
        MODE("Model") AS "Popular Model", -- Most popular EV model in the county
        MODE("Electric Vehicle Type") AS "Popular EV Type", -- Most popular EV type in the county (BEV or PHEV)
        SUM(CASE WHEN "Electric Vehicle Type" = 'Battery Electric Vehicle (BEV)' THEN 1 ELSE 0 END) AS "BEV Count", 
        -- Count of BEVs
        SUM(CASE WHEN "Electric Vehicle Type" = 'Plug-in Hybrid Electric Vehicle (PHEV)' THEN 1 ELSE 0 END) AS "PHEV Count",  
        -- Count of PHEVs
        AVG("income_2023") AS "Median Income",  -- Median income in the county
        AVG("population_2023") AS "Population",  -- Population in the county
        -- Calculating BEV proportion directly within the query
        SUM(CASE WHEN "Electric Vehicle Type" = 'Battery Electric Vehicle (BEV)' THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS "BEV Proportion"  
        -- Proportion of BEVs
    FROM df
    GROUP BY County
    ORDER BY County ASC
""").df()
aggregated_df = aggregated_df.dropna()
aggregated_df.head()

,County,Total EV Count,Average Electric Range,Average Model Year,Popular Brand,Popular Model,Popular EV Type,BEV Count,PHEV Count,Median Income,Population,BEV Proportion
0,Adams,19,131.842105,2018.105263,TESLA,MODEL 3,Battery Electric Vehicle (BEV),10.0,9.0,64498.0,21200.0,0.526316
7,Asotin,41,89.414634,2018.975610,TOYOTA,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),17.0,24.0,67820.0,22650.0,0.414634
10,Benton,1120,125.328571,2018.716964,TESLA,MODEL 3,Battery Electric Vehicle (BEV),576.0,544.0,87992.0,215500.0,0.514286
21,Chelan,510,142.231373,2018.533333,TESLA,LEAF,Battery Electric Vehicle (BEV),330.0,180.0,84430.0,81500.0,0.647059
23,Clallam,573,110.722513,2018.186736,CHEVROLET,LEAF,Battery Electric Vehicle (BEV),299.0,274.0,68924.0,78075.0,0.521815


In [8]:
# Turn this dataframe into a csv file
aggregated_df.to_csv('Clean_Dataset.csv', index=False)